In [ ]:
import pandas as pd
import numpy as np
import os

os.chdir("/content/drive/MyDrive/ML 대회/MAC")
df = pd.read_csv("./train.csv",index_col=False)
df.head()

,no,name,display,weight(kg),os,core,codename,cpu(GHz),model_year,chipset,used,price
0,0,애플 JH 애플 맥북프로 13형 2019년형 MUHQ2KH/A 무배,13인치(32~35cm),NaN,NaN,NaN,NaN,NaN,2019,NaN,False,1483560
1,1,애플 애플 맥북에어13형 MVFL2KH/A 실버 (주)ipark,13인치(32~35cm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,5204440
2,2,애플 맥북프로 2019 16인치 CTO i7/64GB/1TB CTO,16인치(40~42cm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,3801600
3,3,Apple 맥북 프로 16인치 + 애플케어플러스(선택),16인치(40~42cm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,3190000
4,4,애플 맥북프로 13인치 2019년형 i5/8G/128G/256G,13인치(32~35cm),NaN,NaN,NaN,NaN,NaN,2019,NaN,False,1249200


## 정규표현식을 이용한 동일 상품 찾아내기

(1) 제품 코드를 이용한 Imputation

In [ ]:
import re
b = "애플 JH 애플 맥북프로 13형 2019년형 MUHQ2KH/A"
k = re.findall('(\D\D\D\D\d\D\D/\D)', b)[0]
k

'MUHQ2KH/A'

(2) 동일 상품 찾기

In [ ]:
a = "해외Apple 맥북 프로 13형 2020년형 256G 실버 (배송비,부가세 포함)"
import re
def product(txt):
  if "프로" in txt:   #프로 or 에어
    result = "프로"
  elif "에어" in txt:
    result = "에어"
  else:
    result = "레티나"

  year = re.findall('(\d\d\d\d)', txt)[0]+"년형"  # 모델 년도 
  if "인치" in txt:
    inch = re.findall('(\d\d)인치', txt)[0] + "인치"
  else:
    inch = re.findall('(\d\d)형', txt)[0] +"인치"

  result = result + year + inch
  return result
product(a)

'프로2020년형13인치'

In [ ]:
import re
def product_num(a):
  res = "."
  tmp = re.findall('(\D\D\D\D\d\D\D/\D)', a)
  if len(tmp) >0 :
    res = tmp[0]
  else:
    res = None
  return res
  
df["pronum"] = df["name"].apply(lambda x: product_num(x))

In [ ]:
def product1(txt):
  if "프로" in txt:   #프로 or 에어
    result = "프로"
  elif "에어" in txt:
    result = "에어"
  else:
    result = "레티나"
  return result

df["proair"] = df["name"].apply(lambda x: product1(x))

In [ ]:
def product2(txt):
  res = "."
  tmp = re.findall('(\d\d\d\d)', txt)
  if len(tmp) >0 :
    res = tmp[0]
  else:
    res = None
  return res

df["mdl_year"] = df["name"].apply(lambda x: product2(x))

In [ ]:
def product3(txt):
  res = "."
  tmp = re.findall('(\d\d\d\d)', txt)
  if len(tmp) >0 :
    res = tmp[0]
  else:
    res = None
  return res

In [ ]:
df.shape

(4671, 15)

In [ ]:
## 조건 1: 제품코드가 같을 경우
## 조건 2: 제품코드가 없다 -> 에어/프로/레티나 + 년형 + 인치 + G